In [1]:
import pandas as pd
import numpy as np
from datetime import datetime


date = datetime.now().strftime('%d-%m-%Y')
# date = '26-05-2025'
date

'17-06-2025'

In [2]:
ev = pd.read_csv(f"./input/{date}/new-ev.csv", dtype=str)
fdm = pd.read_csv(
    f"./input/{date}/fleet-device-mapping.csv", dtype=str)


# Convert column names to snake case and remove special characters
ev.columns = ev.columns.str.lower().str.replace(
    '[^a-zA-Z0-9]', '_', regex=True).str.replace('__+', '_', regex=True).str.strip('_').str.strip()
fdm.columns = fdm.columns.str.lower().str.replace(
    '[^a-zA-Z0-9]', '_', regex=True).str.replace('__+', '_', regex=True).str.strip('_').str.strip()

In [3]:
ev.info()
fdm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sl_no       120 non-null    object
 1   dep         120 non-null    object
 2   obu_agency  120 non-null    object
 3   fleet       120 non-null    object
 4   iemi_no     120 non-null    object
dtypes: object(5)
memory usage: 4.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3459 entries, 0 to 3458
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dep             3459 non-null   object
 1   fleet           3459 non-null   object
 2   type            3457 non-null   object
 3   regndate        3459 non-null   object
 4   obu_iemi        3198 non-null   object
 5   chalo_deviceid  1216 non-null   object
 6   date            3459 non-null   object
 7   vehicle_number  2579 non-null   object
 8   depot           2579 non

In [7]:
ev["sl_no"].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
115    False
116    False
117    False
118    False
119    False
Name: sl_no, Length: 120, dtype: bool

In [4]:
merged = pd.merge(fdm, ev, on='fleet', how='outer')
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3506 entries, 0 to 3505
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dep_x           3459 non-null   object
 1   fleet           3506 non-null   object
 2   type            3457 non-null   object
 3   regndate        3459 non-null   object
 4   obu_iemi        3198 non-null   object
 5   chalo_deviceid  1216 non-null   object
 6   date            3459 non-null   object
 7   vehicle_number  2579 non-null   object
 8   depot           2579 non-null   object
 9   ny_device_id    9 non-null      object
 10  sl_no           120 non-null    object
 11  dep_y           120 non-null    object
 12  obu_agency      120 non-null    object
 13  iemi_no         120 non-null    object
dtypes: object(14)
memory usage: 383.6+ KB


In [23]:
# check for overlap between fdm.fleet and ev.fleet in merged df. If there is overlap, then check if fdm.obu_iemi, fdm.dep matches with ev.iemi_no, ev.dep. Put the ones that match into overlap_df and others into overlap_mismatch_df
overlap_df = merged[(merged['obu_iemi'] == merged['iemi_no']) & 
                     (merged['dep_x'] == merged['dep_y'])].copy()
overlap_mismatch_df = merged[(merged['obu_iemi'].notna()) & 
                             (~(merged['obu_iemi'] == merged['iemi_no']) |
                               ~(merged['dep_x'] == merged['dep_y']))].copy()

# write the overlap_df and overlap_mismatch_df to csv files
overlap_df.to_csv(f"./output/{date}/overlap_df.csv", index=False)
overlap_mismatch_df.to_csv(f"./output/{date}/overlap_mismatch_df.csv", index=False)

In [15]:
fdm_update = merged.copy()
fdm_update["type_x"] = np.where(merged["type_x"].isna(), merged["type_y"], merged["type_x"])
fdm_update["dep"] = np.where(merged["dep"].isna(),
                           merged["depoot"], merged["dep"])
fdm_update["obu_iemi"] = np.where(merged["obu_iemi"].isna(),
                                merged["imei"], fdm_update["obu_iemi"])

KeyError: 'type_x'

In [31]:
fdm_update.drop(columns=['sl_no', 'depoot', 'type_y', 'imei'], inplace=True)
fdm_update.rename(columns={'type_x': 'type'}, inplace=True)
fdm_update.to_csv(
    f"./output/{date}/fleet-device-mapping_update.csv", index=False)

In [32]:
print(ev['imei'].to_json(orient='values'))


["864650059674926","865860041773202","862607059106913","862607059352053","868142049581045","862607059338508","867032053735614","862607059126572","862607059378066","868142049697478","868142049701288","864977043719461","864977043762180","868142049713788","867032053748344","868142049630057","864513045697361","868142049098195","864650056330043","862607059215128","868142049819841","864977043652001","864513046488885","864513046490030","868142049718803","862607059104041","868142049702641","864513046501760","862607059181775","867383059086100","867032053775040","868142049682587","862607059206077","866041043561965","862607059368968","864650056135814","864513046553282","864977043685613","864977043708803","864977043713191","867032053748666","863418053218475","862607059082411","862607059350685","864650059644515","864650059579000","864513045642441","864977043693443","862607059110345","867383059179509","862607059383512","862607059103449","864513042763281","862607059305853","862607059207604","86260705